In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pm4py==2.1.4

!pip install --upgrade tensorflow==2.2.0

!pip install numpy==1.21.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.19.3-cp38-cp38-manylinux2010_x86_64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
xarray-einstats 0.5.1 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
xarray-einstats 0.5.1 requires scipy>=1.6, but you have scipy 1.4.1 which is incompatible.
plotnine 0.8.0 requires scipy>=1.5.0, but you have scipy 1.4.1 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires scip

In [3]:
import os

In [4]:
#Changing the directory
os.chdir('/content/drive/My Drive/process mining project My Version/Part 2 Project-2')

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import sklearn

from pydream.predictive.nap.NAP import NAP, multiclass_roc_auc_score
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support,precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from pydream.EnhancedPN import EnhancedPN
from pm4py.objects.petri.importer import importer as pnml_importer
from backend.stat.ci_auc import calculate_auc_ci

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
train_json = "tss_heart_log_train0.json"
valid_json ="tss_heart_log_val0.json"
test_json = "tss_heart_log_test0.json"

In [7]:
train_algo = NAP(tss_train_file=train_json, tss_test_file=valid_json, options={"n_epochs" : 2})
train_algo.train(checkpoint_path="chk_points", name="HF_model", save_results=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Xtrain shape: (8403, 1920)
X2train shape: (8403, 3)
Severity train shape: (8403, 58)
X2test shape: (1531, 3)
Xtest shape: (1531, 1920)
Severity test shape: (1531, 58)
Ytrain shape: (8403, 2)
Ytest shape: (1531, 2)
Class weights :  <module 'sklearn.utils.class_weight' from '/usr/local/lib/python3.8/dist-packages/sklearn/utils/class_weight.py'>
Train on 8403 samples, validate on 1531 samples
Epoch 1/2
8403/8403 [==============================] - ETA: 0s - loss: 0.9754 - acc: 0.1801test_prec_mean 0.5801701570680629
test_rec_mean 0.5011664074650077
test_fscore_mean 0.14051125471854561
test_auc_mean 0.5011664074650077

Epoch 00001: test_rec_mean improved from -inf to 0.50117, saving model to chk_points/HF_model_split_weights.hdf5
8403/8403 [==============================] - 13s 1ms/sample - loss: 0.9754 - acc: 0.1801 - val_loss: 1.5984 - val_acc: 0.1620 - test_acc: 0.1620 - test_prec_weighted: 0.8656 - test_rec_weighted: 0.1620 - test_loss: 1.5984 - test_fscore_weighted: 0.0481 - test_auc_w

In [8]:
algo = NAP()
algo.loadModel(path="chk_points", name="HF_model")

X_test, X2_test, severity_test, Y_test = algo.loadData(test_json)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
stdScaler = MinMaxScaler()
stdScaler.fit(X_test)
X_test = stdScaler.transform(X_test)
stdScaler.fit(X2_test)
X2_test = stdScaler.transform(X2_test)
stdScaler.fit(severity_test)
severity_test = stdScaler.transform(severity_test)

Y_test_int = np.array([int(i) for i in Y_test])
y_prob = algo.model.predict([X_test,X2_test,severity_test])
y_pred = np.argmax(y_prob, axis=1)

test_acc = accuracy_score(Y_test_int, y_pred, normalize=True)

In [10]:
y_true =Y_test_int
y_probscore = []
list_of_lists = []
for x in y_prob:
    list_of_lists.append(list(x))
    y_probscore.append(list(x)[1])


y_score = np.array(y_probscore)
y_pred = np.around(y_pred)
y_true = np.array(y_true)
auc, auc_cov, ci = calculate_auc_ci(y_true,y_score,y_pred, alpha=0.95)
print("AUC Score: ",auc)
print("AUC_cov: ", auc_cov)
print("confidence interval: ",ci)

*** Confusion Matrix ***
true\pred     0     1 
        0   1.0 236.0 
        1   1.0  56.0 

AUC Score:  0.590717299578059
AUC_cov:  0.001936534133633672
confidence interval:  [0.50446699 0.67696761]
